In [1]:
import os
import json
from glob import glob
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from datetime import datetime, timedelta

from google.cloud import bigquery
from google.oauth2 import service_account

pd.set_option('display.max_seq_items', None)
pd.set_option('display.max_columns', None)

In [2]:
credentials = service_account.Credentials.from_service_account_file(r'본인추가')
project_id = "본인추가"

In [3]:
client = bigquery.Client(credentials = credentials, project = project_id)
dataset_ref = client.dataset('google_analytics_sample', project='bigquery-public-data')

dataset = client.get_dataset(dataset_ref)

In [4]:
# 데이터셋을 테이블 단위로 보기
tables = list(client.list_tables(dataset))
table_names = sorted([t.table_id for t in tables])

# 테이블 단위로 간단한 정보 확인
print(f"""table 개수 : {len(tables)}
tables : {", ".join(table_names[:3])}, ...
date 범위 : {table_names[0][-8:]} ~ {table_names[-1][-8:]}""")

table 개수 : 366
tables : ga_sessions_20160801, ga_sessions_20160802, ga_sessions_20160803, ...
date 범위 : 20160801 ~ 20170801


In [5]:
# 테이블 경로 생성
table_ref_temp = dataset_ref.table(table_names[0])

# 테이블 가져오기
table_temp = client.get_table(table_ref_temp)

# 컬럼 확인
client.list_rows(table_temp, max_results=2).to_dataframe()

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,userId,channelGrouping,socialEngagementType
0,<NA>,1,1470046245,1470046245,20160801,"{'visits': 1, 'hits': 24, 'pageviews': 17, 'ti...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Firefox', 'browserVersion': 'not ...","{'continent': 'Europe', 'subContinent': 'Weste...","[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 3, 'minut...",895954260133011192,None,Organic Search,Not Socially Engaged
1,<NA>,1,1470084717,1470084717,20160801,"{'visits': 1, 'hits': 24, 'pageviews': 18, 'ti...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Internet Explorer', 'browserVersi...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 13, 'minu...",0288478011259077136,None,Direct,Not Socially Engaged


In [17]:
def format_schema_field(schema_field, indent=0):
    """
    빅쿼리 스키마의 (중첩된 구조 내부까지) 필드 이름과 데이터 타입을 출력하는 함수
    """
    
    indent_str = "  " * indent
    field_info = f"{indent_str}{schema_field.name} ({schema_field.field_type})"
    
    if schema_field.mode != "NULLABLE":
        field_info += f" - {schema_field.mode}"
    
    if schema_field.description:
        field_info += f" - {schema_field.description}"
    
    nested_indent = indent + 2
    if schema_field.field_type == "RECORD":
        for sub_field in schema_field.fields:
            field_info += "\n" + format_schema_field(sub_field, nested_indent)
    
    return field_info

# Display schemas
print("SCHEMA field for the 'totals' column:\n")
print(format_schema_field(table_temp.schema[5]))
print()

print("\nSCHEMA field for the 'trafficSource' column:\n")
print(format_schema_field(table_temp.schema[6]))
print()

print("\nSCHEMA field for the 'device' column:\n")
print(format_schema_field(table_temp.schema[7]))
print()

print("\nSCHEMA field for the 'geoNetwork' column:\n")
print(format_schema_field(table_temp.schema[8]))
print()

print("\nSCHEMA field for the 'customDimensions' column:\n")
print(format_schema_field(table_temp.schema[9]))
print()

print("\nSCHEMA field for the 'hits' column:\n")
print(format_schema_field(table_temp.schema[10]))

SCHEMA field for the 'totals' column:

totals (RECORD)
    visits (INTEGER)
    hits (INTEGER)
    pageviews (INTEGER)
    timeOnSite (INTEGER)
    bounces (INTEGER)
    transactions (INTEGER)
    transactionRevenue (INTEGER)
    newVisits (INTEGER)
    screenviews (INTEGER)
    uniqueScreenviews (INTEGER)
    timeOnScreen (INTEGER)
    totalTransactionRevenue (INTEGER)
    sessionQualityDim (INTEGER)


SCHEMA field for the 'trafficSource' column:

trafficSource (RECORD)
    referralPath (STRING)
    campaign (STRING)
    source (STRING)
    medium (STRING)
    keyword (STRING)
    adContent (STRING)
    adwordsClickInfo (RECORD)
        campaignId (INTEGER)
        adGroupId (INTEGER)
        creativeId (INTEGER)
        criteriaId (INTEGER)
        page (INTEGER)
        slot (STRING)
        criteriaParameters (STRING)
        gclId (STRING)
        customerId (INTEGER)
        adNetworkType (STRING)
        targetingCriteria (RECORD)
            boomUserlistId (INTEGER)
        isV

In [92]:
final = """
SELECT
    fullVisitorId,
    visitId,
    hitNumber as hits_hitNumber,
    geoNetwork.region as geoNetwork_region,
    geoNetwork.metro as geoNetwork_metro,
    geoNetwork.city as geoNetwork_city,
    latencyTracking.domContentLoadedTime as hits_latencyTracking_domContentLoadedTime,
    latencyTracking.domInteractiveTime as hits_latencyTracking_domInteractiveTime,
    latencyTracking.pageDownloadTime as hits_latencyTracking_pageDownloadTime
FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
    UNNEST(hits)
WHERE
    _TABLE_SUFFIX BETWEEN '20160801' AND '20170730' and
    geoNetwork.country = 'United States'
"""

final = client.query(final).result().to_dataframe()
final


,fullVisitorId,visitId,hits_hitNumber,geoNetwork_region,geoNetwork_metro,geoNetwork_city,hits_latencyTracking_domContentLoadedTime,hits_latencyTracking_domInteractiveTime,hits_latencyTracking_pageDownloadTime
0,0450212169679268027,1495208012,1,California,San Francisco-Oakland-San Jose CA,Mountain View,<NA>,<NA>,<NA>
1,5033770040134723177,1495218204,2,not available in demo dataset,not available in demo dataset,not available in demo dataset,<NA>,<NA>,<NA>
2,3611868059967146894,1495253252,3,California,San Francisco-Oakland-San Jose CA,San Francisco,<NA>,<NA>,<NA>
3,1600893372402587161,1495206222,4,not available in demo dataset,not available in demo dataset,not available in demo dataset,<NA>,<NA>,<NA>
4,7445235885559107095,1495224397,3,not available in demo dataset,not available in demo dataset,not available in demo dataset,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...
2464868,8950151145554417908,1497962761,2,Georgia,Atlanta GA,Atlanta,<NA>,<NA>,<NA>
2464869,1539879642689449498,1498008415,1,not available in demo dataset,not available in demo dataset,not available in demo dataset,<NA>,<NA>,<NA>
2464870,5948467132415991902,1498021664,2,not available in demo dataset,not available in demo dataset,not available in demo dataset,<NA>,<NA>,<NA>
2464871,361348493021521937,1498016272,1,California,Los Angeles CA,Los Angeles,<NA>,<NA>,<NA>


In [93]:
final.to_csv('geo_latencyTracking.csv', index=False)

In [7]:
# # 판매 및 비판매 거래 채널 그룹별 거래 수 조회
# query = """
# SELECT
#     product.v2ProductName as hits_product_v2ProductName,
#     product.v2ProductCategory as hits_product_v2ProductCategory,
#     product.productPrice as hits_product_productPrice,
#     product.productQuantity as hits_product_productQuantity,
#     product.productRevenue as hits_product_productRevenue,
#     product.localProductRevenue as hits_product_localProductRevenue,
#     promotion.promoId as hits_promotion_promoId,
#     promotion.promoName as hits_promotion_promoName,
#     promotion.promoCreative as hits_promotion_promoCreative,
#     promotion.promoPosition as hits_promotion_promoPosition
# FROM
#     `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
#     UNNEST(hits) AS hits,
#     UNNEST(hits.product) AS product,
#     UNNEST(hits.promotion) AS promotion
# WHERE
#     _TABLE_SUFFIX BETWEEN '20160801' AND '20170730' and
#     geoNetwork.country = 'United States'
# """

# query = client.query(query).to_dataframe()
# query


In [29]:
# 판매 및 비판매 거래 채널 그룹별 거래 수 조회
query = """
SELECT
    fullVisitorId,
    visitId,
    visitNumber,
    hitNumber as hits_hitNumber,
    transaction.transactionRevenue as hits_transaction_transactionRevenue,
    eCommerceAction.action_type as hits_eCommerceAction_action_type,
    v2ProductName as hits_product_v2ProductName,
    v2ProductCategory as hits_product_v2ProductCategory,
    productPrice as hits_product_productPrice,
    productQuantity as hits_product_productQuantity,
    productRevenue as hits_product_productRevenue,
    productRefundAmount as hits_product_productRefundAmount,
    productListPosition as hits_product_productListPosition,
    productCouponCode as hits_product_productCouponCode,
    productListName as hits_product_productListName,
    isClick as hits_product_isClick
FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
    UNNEST(hits) AS hits,
    UNNEST(product) AS product
WHERE
    _TABLE_SUFFIX BETWEEN '20160801' AND '20170730' and
    geoNetwork.country = 'United States'
limit 5
"""

query = client.query(query).result().to_dataframe()
query


,fullVisitorId,visitId,visitNumber,hits_hitNumber,hits_transaction_transactionRevenue,hits_eCommerceAction_action_type,hits_product_v2ProductName,hits_product_v2ProductCategory,hits_product_productPrice,hits_product_productQuantity,hits_product_productRevenue,hits_product_productRefundAmount,hits_product_productListPosition,hits_product_productCouponCode,hits_product_productListName,hits_product_isClick
0,8722180542488133094,1477531953,1,1,<NA>,0,Google Men's 100% Cotton Short Sleeve Hero Tee...,Home/Apparel/Men's/Men's-T-Shirts/,16990000,<NA>,<NA>,<NA>,1,None,Category,<NA>
1,8722180542488133094,1477531953,1,1,<NA>,0,Google Men's 100% Cotton Short Sleeve Hero Tee...,Home/Apparel/Men's/Men's-T-Shirts/,16990000,<NA>,<NA>,<NA>,2,None,Category,<NA>
2,8722180542488133094,1477531953,1,1,<NA>,0,Google Men's 100% Cotton Short Sleeve Hero Tee...,Home/Apparel/Men's/Men's-T-Shirts/,16990000,<NA>,<NA>,<NA>,3,None,Category,<NA>
3,8722180542488133094,1477531953,1,1,<NA>,0,Google Heavyweight Long Sleeve Hero Tee Burgundy,Home/Apparel/Men's/Men's-T-Shirts/,39990000,<NA>,<NA>,<NA>,4,None,Category,<NA>
4,8722180542488133094,1477531953,1,1,<NA>,0,Google Heavyweight Long Sleeve Hero Tee Navy,Home/Apparel/Men's/Men's-T-Shirts/,39990000,<NA>,<NA>,<NA>,5,None,Category,<NA>


In [10]:
# # 판매 및 비판매 거래 채널 그룹별 거래 수 조회
# # 손상금지
# query = """
# SELECT
#     channelGrouping as channelGrouping,
#     fullVisitorId as fullVisitorId,
#     visitId as visitId,
#     visitNumber as visitNumber,
#     visitStartTime as visitStartTime,
#     totals.sessionQualityDim as totals_sessionQualityDim,
#     totals.newVisits as totals_newVisits,
#     totals.totalTransactionRevenue as totals_totalTransactionRevenue,
#     totals.bounces as totals_bounces,
#     trafficSource.referralPath as trafficSource_referralPath,
#     trafficSource.campaign as trafficSource_campaign,
#     trafficSource.medium as trafficSource_medium,
#     trafficSource.keyword as trafficSource_keyword,
#     trafficSource.adContent as trafficSource_adContent,
#     trafficSource.isTrueDirect as trafficSource_isTrueDirect,
#     trafficSource.adwordsClickInfo_page as trafficSource_adwordsClickInfo_page,
#     trafficSource.adwordsClickInfo_slot as trafficSource_adwordsClickInfo_slot,
#     hits.hitNumber as hits_hitNumber,
#     hits.hour as hits_hour,
#     hits.minute as hits_minute,
#     hits.eventInfo as hits_eventInfo,
#     hits.type as hits_type,
#     hits.referer as hits_referer,
#     hits.isExit as hits_isExit,
#     hits.page.pageTitle as hits_page_pageTitle,
#     hits.page.searchKeyword as hits_page_searchKeyword,
#     hits.transaction.transactionRevenue as hits_transaction_transactionRevenue,
#     hits.transaction.transactionTax as hits_transaction_transactionTax,
#     hits.transaction.transactionShipping as hits_transaction_transactionShipping,
#     hits.transaction.localTransactionRevenue as hits_transaction_localTransactionRevenue,
#     hits.transaction.localTransactionTax as hits_transaction_localTransactionTax,
#     hits.transaction.localTransactionShipping as hits_transaction_localTransactionShipping,
#     hits.eCommerceAction.action.type as hits_eCommerceAction_action_type,
#     hits.eCommerceAction.step as hits_eCommerceAction_step,
#     hits.social.socialNetwork as hits_social_socialNetwork,
#     hits.social.hasSocialSourceReferral as hits_social_hasSocialSourceReferral,
#     hits.product.v2ProductName as hits_product_v2ProductName,
#     hits.product.v2ProductCategory as hits_product_v2ProductCategory,
#     hits.product.productPrice as hits_product_productPrice,
#     hits.product.productQuantity as hits_product_productQuantity,
#     hits.product.productRevenue as hits_product_productRevenue,
#     hits.product.localProductRevenue as hits_product_localProductRevenue,
#     hits.promotion.promoId as hits_promotion_promoId,
#     hits.promotion.promoName as hits_promotion_promoName,
#     hits.promotion.promoCreative as hits_promotion_promoCreative,
#     hits.promotion.promoPosition as hits_promotion_promoPosition
# FROM
#     `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
#     UNNEST(hits) AS hits
# WHERE
#     _TABLE_SUFFIX BETWEEN '20160801' AND '20170730' and
#     geoNetwork.country == 'United States'
# """


In [11]:
# # 쿼리 실행 및 결과 가져오기
# query = client.query(query).to_dataframe()